In [1]:
import coiled

cluster = coiled.Cluster(n_workers=2, backend_options={"region_name": "us-east-1"})
cluster.adapt(minimum=10, maximum=200)
client = cluster.get_client()

Output()

Output()

2023-08-18 12:35:55,259 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=10 maximum=200


In [2]:
client

<Client: 'tls://10.0.90.174:8786' processes=2 threads=8, memory=29.68 GiB>

## Setup

In [3]:
%load_ext watermark

import flox  # make sure its available
import fsspec
import numpy as np
import rioxarray
import xarray as xr

xr.set_options(
    display_expand_attrs=False,
    display_expand_coords=False,
    display_expand_data=True,
)

%watermark -iv

xarray   : 2023.6.0
coiled   : 0.8.10
fsspec   : 2023.6.0
flox     : 0.7.2
rioxarray: 0.14.1
numpy    : 1.24.4



## Load NWM data

In [5]:
ds = xr.open_zarr(
    fsspec.get_mapper(
        "s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr", anon=True
    ),
    consolidated=True,
)
ds

<xarray.Dataset>
Dimensions:     (feature_id: 2776738, time: 367439)
Coordinates: (7)
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes: (5)

In [11]:
%%time

resampler = ds.resample(time="A-SEP", label="right", skipna=True)
# Get the maximum value for each data variable in the dataset
ds_WY_max = resampler.max()
ds_WY_max = ds_WY_max.rename(
    {"streamflow": "streamflow_max", "velocity": "velocity_max"}
)

# Get the minimum value for each data variable in the dataset
ds_WY_min = resampler.min()
ds_WY_min = ds_WY_min.rename(
    {"streamflow": "streamflow_min", "velocity": "velocity_min"}
)

ds_out = xr.merge([ds_WY_max, ds_WY_min], compat="override")
ds_out

CPU times: user 3.72 s, sys: 167 ms, total: 3.89 s
Wall time: 9.6 s


<xarray.Dataset>
Dimensions:         (time: 43, feature_id: 2776738)
Coordinates: (7)
Data variables:
    streamflow_max  (time, feature_id) float64 dask.array<chunksize=(1, 30000), meta=np.ndarray>
    velocity_max    (time, feature_id) float64 dask.array<chunksize=(1, 30000), meta=np.ndarray>
    crs             (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    streamflow_min  (time, feature_id) float64 dask.array<chunksize=(1, 30000), meta=np.ndarray>
    velocity_min    (time, feature_id) float64 dask.array<chunksize=(1, 30000), meta=np.ndarray>
Attributes: (5)

In [ ]:
ds_out.to_zarr("s3://cherian-scratch/nwm-streamflow-water-year.zarr", mode="w")

/Users/dcherian/mambaforge/envs/coiled/lib/python3.11/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 35.73 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
